In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# Getting Started with Merlin Models: Develop a Model for MovieLens

## Overview

[Merlin Models](https://github.com/NVIDIA-Merlin/models/) is a library for training recommender models. Merlin Models let Data Scientists and ML Engineers easily train standard RecSys models on their own dataset, getting GPU-accelerated models with best practices baked into the library. This will also let researchers to build custom models by incorporating standard components of deep learning recommender models, and then benchmark their new models on example offline datasets. Merlin Models is part of the [Merlin open source framework](https://developer.nvidia.com/nvidia-merlin).

Core features are:
- Many different recommender system architectures (tabular, two-tower, sequential) or tasks (binary, multi-class classification, multi-task)
- Flexible APIs targeted to both production and research
- Deep integration with NVIDIA Merlin platform, including NVTabular for ETL and Merlin Systems model serving


### Learning objectives

- Training [Facebook's DLRM model](https://arxiv.org/pdf/1906.00091.pdf) very easily with our high-level API.
- Understanding Merlin Models high-level API

## Downloading and preparing the dataset

In [2]:
import merlin.models.tf as mm

from merlin.models.data.movielens import get_movielens
from merlin.schema.tags import Tags

2022-03-30 16:24:05.620762: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-30 16:24:07.820943: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16249 MB memory:  -> device: 0, name: Quadro GV100, pci bus id: 0000:15:00.0, compute capability: 7.0
2022-03-30 16:24:07.822085: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 30611 MB memory:  -> device: 1, name: Quadro GV100, pci bus id: 0000:2d:00.0, compute capability: 7.0
/home/gmoreira/projects/nvidia/nvidia_merlin/NVTabular/nvtabular/graph.py:23: FutureWarning: The `nvtabular.graph` module has moved to `merlin.dag

We provide the `get_movielens()` function as a convenience to download the dataset, perform simple preprocessing, and split the data into training and validation datasets.

In [3]:
train, valid = get_movielens(variant="ml-1m")

/home/gmoreira/miniconda3/envs/merlin_22.04_dev/lib/python3.8/site-packages/cudf/core/dataframe.py:1253: UserWarning: The deep parameter is ignored and is only included for pandas compatibility.
  warnings.warn(


## Training the DLRM Model with Merlin Models

We define the DLRM model, whose prediction task is a binary classification. From the `schema`, the categorical features are identified (and embedded) and the target column is also automatically inferred, because of the schema tags. We talk more about the schema in the example notebook [02-Merlin-Models-and-NVTabular-applying-to-your-own-dataset](02-Merlin-Models-and-NVTabular-applying-to-your-own-dataset),

In [4]:
model = mm.DLRMModel(
    train.schema,
    embedding_dim=64,
    bottom_block=mm.MLPBlock([128, 64]),
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(
        train.schema.select_by_tag(Tags.TARGET).column_names[0]
    ),
)

model.compile(optimizer="adam")

Next, we train the model.

In [5]:
model.fit(train, batch_size=1024)

2022-03-30 16:24:08.840469: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


782/782 [==============================] - 10s 10ms/step - rating_binary/binary_classification_task/precision: 0.7113 - rating_binary/binary_classification_task/recall: 0.8239 - rating_binary/binary_classification_task/binary_accuracy: 0.7065 - rating_binary/binary_classification_task/auc: 0.7682 - loss: 0.5637 - regularization_loss: 0.0000e+00 - total_loss: 0.5637


We evaluate the model...

In [6]:
metrics = model.evaluate(valid, batch_size=1024, return_dict=True)

  1/196 [..............................] - ETA: 2:25 - rating_binary/binary_classification_task/precision: 0.7307 - rating_binary/binary_classification_task/recall: 0.8037 - rating_binary/binary_classification_task/binary_accuracy: 0.7109 - rating_binary/binary_classification_task/auc: 0.7813 - loss: 0.5439 - regularization_loss: 0.0000e+00 - total_loss: 0.5439

2022-03-30 16:24:20.062458: W tensorflow/core/grappler/optimizers/loop_optimizer.cc:907] Skipping loop optimization for Merge node with control input: cond/then/_0/cond/cond/branch_executed/_101


196/196 [==============================] - 2s 7ms/step - rating_binary/binary_classification_task/precision: 0.7329 - rating_binary/binary_classification_task/recall: 0.8211 - rating_binary/binary_classification_task/binary_accuracy: 0.7247 - rating_binary/binary_classification_task/auc: 0.7884 - loss: 0.5430 - regularization_loss: 0.0000e+00 - total_loss: 0.5430


... and check the evaluation metrics. We use by default typical binary classification metrics -- Precision, Recall, Accuracy and AUC. But you can also provide your own metrics list by setting `BinaryClassificationTask(..., metrics=[])`.

In [7]:
metrics

{'rating_binary/binary_classification_task/precision': 0.7329489588737488,
 'rating_binary/binary_classification_task/recall': 0.8210552930831909,
 'rating_binary/binary_classification_task/binary_accuracy': 0.7246914505958557,
 'rating_binary/binary_classification_task/auc': 0.7884241342544556,
 'loss': 0.5135267972946167,
 'regularization_loss': 0.0,
 'total_loss': 0.5135267972946167}

## Conclusion

Merlin Models enables users to define and train a deep learning recommeder model with only 3 commands.

```python
model = mm.DLRMModel(
    train.schema,
    embedding_dim=64,
    bottom_block=mm.MLPBlock([128, 64]),
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(
        train.schema.select_by_tag(Tags.TARGET).column_names[0]
    ),
)
model.compile(optimizer="adam")
model.fit(train, batch_size=1024)
```

## Next steps

In the next example notebooks, we will show how the integration with NVTabular and how to explore different recommender models.